In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=6c885f7fec5bbc7ceea3201b7dfe9da12b95657157215d6ed475fab1036b20f8
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=57cdd261b40e01aedb82c2c295aa4d8e1555c72d131a8b59ce3f2ed0297af7e6
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
fashion_mnist=keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [11]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [21]:
def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                              kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
                              activation='relu',
                              input_shape=(28,28,1)

                          ),
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                              kernel_size=hp.Choice('conv_2_kernal',values=[3,5]),
                              activation='relu'
                          ),
                          keras.layers.Flatten(),
                          keras.layers.Dense(
                              units=hp.Int('dence_1_units',min_value=32,max_value=128,step=16),
                              activation='relu'
                          ),
                          keras.layers.Dense(
                            10,activation='softmax'
                          )


  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [22]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [23]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 18m 22s]
val_accuracy: 0.9016666412353516

Best val_accuracy So Far: 0.9088333249092102
Total elapsed time: 01h 03m 19s
INFO:tensorflow:Oracle triggered exit


In [25]:
model=tuner_search.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2457728   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,576,762
Trainable params: 2,576,762
Non-trainable params: 0
_________________________________________________________________
